In [0]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from skimage import io, transform
import numpy as np

plt.ion()   # interactive mode

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
#!rm sample_data/*.jpg # I needed to remove some old images from current directory. Keep for now. 

In [1]:
# !mkdir data
#mv *.jpg data/
!mkdir data/train
!mkdir data/val
!mkdir data/train/soccer_ball
!mkdir data/val/soccer_ball

In [4]:
# in google colab this shell script works with %%shell. In plain jupyter notebook works with %%bash.

# %%bash
%%shell
#!/bin/bash  
a=0
for i in data/*.jpg; do
  mv -- "$i" "data/train/soccer_ball"
  let a=a+1
  if [ $a -eq 350 ]; then
    break
  fi
done

In [5]:
# %%bash
%%shell
#!/bin/bash  
a=0
for i in data/*.jpg; do
  mv -- "$i" "data/val/soccer_ball"
  let a=a+1
  if [ $a -eq 100 ]; then
    break
  fi
done

In [0]:
!pip install --no-cache-dir -I pillow
%reload_ext autoreload   
%autoreload 0  

!pip install Pillow==4.0.0
!pip install PIL
!pip install image

In [138]:
torch.cuda.is_available()

True

In [0]:
class FifaBallDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, list_IDs, labels, transform=None):
        self.list_IDs = list_IDs
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.list_IDs)

    def __getitem__(self, idx):
        id = self.list_IDs[idx]
        img_name =  os.path.join('data/' , str(id) + '.jpg')
        image = io.imread(img_name)

        y = self.labels[idx] ### It does not matter in our case, all labels are 1
        
        if self.transform:
            image = self.transform(image)
       
        return image, y

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Comment the cell below 

In [0]:
##### comment below ########
np.random.seed(1)
dataset_size = 450
ids = np.arange(1,dataset_size+1)
np.random.shuffle(ids)

# Parameters
params = {'batch_size': 50,
          'shuffle': True,
          'num_workers': 4, }

train_size = 350 
valid_size = 100


training_set = FifaBallDataset(ids[:train_size], np.zeros(train_size),
                                           transform=transforms.Compose([
                                            transforms.ToPILImage(),
                                            transforms.Resize(256),
                                            transforms.RandomResizedCrop(224),
                                            transforms.RandomHorizontalFlip(),
                                            transforms.ToTensor()
                                           ]))

trainloader = DataLoader(training_set, **params)

validation_set = FifaBallDataset(ids[train_size:train_size+valid_size], np.zeros(valid_size),
                                           transform=transforms.Compose([
                                        transforms.ToPILImage(),
                                            transforms.Resize(256),
                                            transforms.RandomResizedCrop(224),
                                            transforms.RandomHorizontalFlip(),
                                            transforms.ToTensor()
                                           ]))

validloader = DataLoader(validation_set, **params)

In [0]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
    ]),
}

data_dir = 'data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [170]:
len(image_datasets['train'])

350